# 📊 PISA 2022 UK Data Preprocessing for Tree-Based Models

This notebook prepares the **PISA 2022 UK dataset** for training tree-based machine learning models, specifically **Decision Trees**, **Random Forests**, and **XGBoost**. The preprocessing steps are tailored to suit these models, which do not require feature scaling but handle categorical and missing data in specific ways.

### 🔧 Key Steps:
1. **Reverse-Code Selected Variables** – Adjusts the scale of items where higher values originally indicated more negative outcomes.
2. **Binary Variable Creation** – Recodes key categorical variables (e.g., gender, computer access) into binary format.
3. **Variable Renaming** – Renames columns with meaningful, interpretable names across categories (e.g., individual, academic, family, school, peer/social, contextual).
4. **Missing Data Handling**  
   - Drops variables with >30% missing values.  
   - Drops rows missing the target variable (`math_1`).  
   - Uses **median imputation** for numeric features and **mode imputation** for binary/categorical features.
5. **Data Splitting** – Splits the dataset into training, validation, and test sets (60/20/20 split).
6. **One-Hot Encoding** – Applies to the `region` variable and aligns encoded columns across all datasets.
7. **Export Clean Data** – Saves preprocessed training, validation, and test sets to CSV files for downstream modeling.

> **Note:** Exploratory Data Analysis (EDA) was conducted separately and is not included in this notebook.


# 0. Import libraries and load data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("/workspaces/mini_project_2/data/pisa_2022_uk_selected.csv")

In [3]:
df.head()

,CNTSTUID,W_FSTUWT,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV6MATH,PV7MATH,PV8MATH,...,ST270Q02JA,ST038Q05NA,ST265Q03JA,ST272Q01JA,ST034Q02TA,ST038Q03NA,ST038Q04NA,ST315Q04JA,ST315Q06JA,REGION
0,82600003.0,4.77923,699.809,598.369,593.952,603.361,666.143,635.207,608.553,583.002,...,4.0,1.0,1.0,4.0,NaN,1.0,1.0,NaN,NaN,82613.0
1,82600004.0,8.23968,454.479,377.041,463.036,394.994,396.957,429.649,459.601,439.692,...,1.0,1.0,2.0,8.0,2.0,2.0,2.0,NaN,NaN,82612.0
2,82600005.0,136.31080,566.143,616.946,556.976,656.532,652.062,611.077,657.387,645.288,...,1.0,1.0,1.0,10.0,2.0,1.0,1.0,4.0,3.0,82611.0
3,82600006.0,84.42297,371.820,357.674,264.029,393.236,361.008,326.822,390.370,338.591,...,3.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,NaN,82611.0
4,82600007.0,110.62140,423.607,382.887,414.527,404.693,388.663,427.517,447.633,445.616,...,2.0,1.0,1.0,8.0,1.0,2.0,2.0,NaN,4.0,82611.0


## 1. Recode variables 

In [4]:
# Reverse-coded variables with their original scales
# Format: "VARIABLE_NAME": (min_value, max_value)
reverse_vars = {
    "ST309Q02JA": (1, 5),  # not_distract: 1–5
    "ST258Q01JA": (1, 5),  # food_sec: 1–5
    "ST034Q03TA": (1, 4),  # schl_belong: 1–4
    "ST270Q02JA": (1, 4),  # teacher_help: 1–4
    "ST038Q05NA": (1, 4),  # safe_student: 1–4
    "ST265Q03JA": (1, 4),  # safe_class: 1–4
    "ST034Q02TA": (1, 4),  # make_friends: 1–4
    "ST038Q03NA": (1, 4),  # feel_included: 1–4
    "ST038Q04NA": (1, 4)   # no_mock: 1–4
}

In [5]:
def reverse_code(df, reverse_map):
    df = df.copy()
    for var, (min_val, max_val) in reverse_map.items():
        if var in df.columns:
            df[var] = max_val + min_val - df[var]
    return df

df = reverse_code(df, reverse_vars)

## 2. Create Binary 

In [6]:
df["ST004D01T"] = df["ST004D01T"].apply(lambda x: 1 if x == 1 else 0 if x == 2 else np.nan)  # gender
df["ST327Q06JA"] = df["ST327Q06JA"].apply(lambda x: 1 if x == 1 else 0 if x == 2 else np.nan)  # expt_bach
df["ST250Q02JA"] = df["ST250Q02JA"].apply(lambda x: 1 if x == 1 else 0 if x == 2 else np.nan)  # has_computer
df["ST255Q01JA"] = df["ST255Q01JA"].apply(lambda x: 1 if x == 1 else 0 if x in [2,3,4,5,6,7] else np.nan)  # has_books

In [7]:
df[["ST004D01T", "ST327Q06JA", "ST250Q02JA", "ST255Q01JA"]].head()

,ST004D01T,ST327Q06JA,ST250Q02JA,ST255Q01JA
0,0,1.0,1.0,0.0
1,0,0.0,1.0,0.0
2,0,1.0,1.0,0.0
3,1,NaN,0.0,0.0
4,0,NaN,1.0,0.0


## 3. Rename variables 

In [8]:
# Core renaming dictionary
rename_dict = {
    # Individual
    "ST004D01T": "gender",
    "ST016Q01NA": "life_sat",
    "ST313Q01JA": "emo_control",
    "ST309Q02JA": "not_distract",
    "ST301Q01JA": "curiosity",
    "ST305Q01JA": "comfort_lead",

    # Academic
    "ST296Q01JA": "math_hwork",
    "ST293Q03JA": "math_effort",
    "ST292Q01JA": "math_conf",
    "ST327Q06JA": "expt_bach",
    "ST355Q05JA": "conf_self_mot",

    # Family
    "ESCS": "SES",
    "ST230Q01JA": "num_sib",
    "ST258Q01JA": "food_sec",
    "ST259Q01JA": "family_stat",
    "ST300Q01JA": "parent_talk_schl",
    "ST300Q02JA": "parent_eat_with",
    "ST250Q02JA": "has_computer",
    "ST255Q01JA": "has_books",

    # School
    "ST034Q03TA": "schl_belong",
    "ST267Q01JA": "teach_respect",
    "ST267Q05JA": "teac_interest",
    "ST273Q01JA": "listen_teacher",
    "ST285Q04JA": "teacher_explain",
    "ST270Q02JA": "teacher_help",
    "ST038Q05NA": "safe_student",
    "ST265Q03JA": "safe_class",
    "ST272Q01JA": "qual_math_instruct",

    # Peer/Social
    "ST034Q02TA": "make_friends",
    "ST038Q03NA": "feel_included",
    "ST038Q04NA": "no_mock",
    "ST315Q04JA": "trust_friends",
    "ST315Q06JA": "trust_gen",

    # Contextual
    "REGION": "region"
}

In [9]:
# Add PV renaming (PV1MATH → math_1, PV1READ → read_1, etc.)
for subject in ["MATH", "READ", "SCIE"]:
    for i in range(1, 11):
        old = f"PV{i}{subject}"
        new = f"{subject.lower()}_{i}"
        rename_dict[old] = new
df.rename(columns=rename_dict, inplace=True)

In [10]:
# Map region codes 
region_map = {
    82611: "england",
    82612: "n_ireland",
    82613: "wales",
    82620: "scotland"
}

df["region"] = df["region"].map(region_map)

In [11]:
# Create binary target from math_1 using the median
math_median = df["math_1"].median()
df["math_binary"] = df["math_1"].apply(lambda x: 1 if x >= math_median else 0)

print(df["math_binary"].value_counts())


math_binary
1    6486
0    6486
Name: count, dtype: int64


In [12]:
print("Renamed columns:", df.columns.tolist())

Renamed columns: ['CNTSTUID', 'W_FSTUWT', 'math_1', 'math_2', 'math_3', 'math_4', 'math_5', 'math_6', 'math_7', 'math_8', 'math_9', 'math_10', 'read_1', 'read_2', 'read_3', 'read_4', 'read_5', 'read_6', 'read_7', 'read_8', 'read_9', 'read_10', 'scie_1', 'scie_2', 'scie_3', 'scie_4', 'scie_5', 'scie_6', 'scie_7', 'scie_8', 'scie_9', 'scie_10', 'gender', 'life_sat', 'emo_control', 'not_distract', 'curiosity', 'comfort_lead', 'math_hwork', 'math_effort', 'math_conf', 'expt_bach', 'conf_self_mot', 'SES', 'num_sib', 'food_sec', 'family_stat', 'parent_talk_schl', 'parent_eat_with', 'has_computer', 'has_books', 'schl_belong', 'teach_respect', 'teac_interest', 'listen_teacher', 'teacher_explain', 'teacher_help', 'safe_student', 'safe_class', 'qual_math_instruct', 'make_friends', 'feel_included', 'no_mock', 'trust_friends', 'trust_gen', 'region', 'math_binary']


## 4. Check Missing 

In [13]:
# Percentage of missing values
missing_percent = df.isna().mean() * 100
missing_percent[missing_percent > 0].sort_values(ascending=False)

conf_self_mot         63.737280
parent_talk_schl      61.941104
parent_eat_with       61.856306
comfort_lead          58.872957
not_distract          58.271662
trust_friends         58.240826
trust_gen             58.171446
emo_control           57.747456
curiosity             57.469935
teacher_explain       56.945729
math_effort           56.699044
expt_bach             54.486586
teach_respect         46.068455
teac_interest         45.490287
listen_teacher        40.679926
math_conf             35.368486
schl_belong           28.314832
make_friends          27.952513
math_hwork            18.902251
teacher_help          18.239285
qual_math_instruct    16.412273
family_stat           14.909035
SES                   14.554425
no_mock               14.199815
safe_student          14.161270
feel_included         14.053346
safe_class            13.166821
life_sat              13.120567
food_sec              13.120567
has_computer          12.604070
has_books             11.971940
num_sib 

In [14]:
#check how many rows would be left if dropped all missing 
df_dropped = df.dropna()
print(f"Rows before dropping: {len(df)}")
print(f"Rows after dropping:  {len(df_dropped)}")
print(f"Percentage kept:      {len(df_dropped) / len(df) * 100:.2f}%")

Rows before dropping: 12972
Rows after dropping:  0
Percentage kept:      0.00%


#### drop anythign with more than 30% missing 

In [15]:
to_drop = missing_percent[missing_percent > 30].index.tolist()
df.drop(columns=to_drop, inplace=True)
print(f" Dropped {len(to_drop)} columns with >30% missing")

 Dropped 16 columns with >30% missing


In [16]:
#### drop rows with missing target

In [17]:
df.dropna(subset=["math_1"], inplace=True)

In [18]:
# Define original columns grouped by category
categories = {
    "Individual": [
        "gender", "life_sat", "emo_control", "not_distract", "curiosity", "comfort_lead"
    ],
    "Academic": [
        "math_hwork", "math_effort", "math_conf", "expt_bach", "conf_self_mot"
    ],
    "Family": [
        "SES", "num_sib", "food_sec", "family_stat", "parent_talk_schl", "parent_eat_with", "has_computer", "has_books"
    ],
    "School": [
        "schl_belong", "teach_respect", "teac_interest", "listen_teacher",
        "teacher_explain", "teacher_help", "safe_student", "safe_class", "qual_math_instruct"
    ],
    "Peer/Social": [
        "make_friends", "feel_included", "no_mock", "trust_friends", "trust_gen"
    ],
    "Contextual": [
        "region"
    ]
}

# Filter only those that remain in the dataframe
print("\n✅ Remaining columns by category:\n")
for category, vars in categories.items():
    kept = [var for var in vars if var in df.columns]
    if kept:
        print(f"📂 {category}:")
        for col in kept:
            print(f"   - {col}")
        print()


✅ Remaining columns by category:

📂 Individual:
   - gender
   - life_sat

📂 Academic:
   - math_hwork

📂 Family:
   - SES
   - num_sib
   - food_sec
   - family_stat
   - has_computer
   - has_books

📂 School:
   - schl_belong
   - teacher_help
   - safe_student
   - safe_class
   - qual_math_instruct

📂 Peer/Social:
   - make_friends
   - feel_included
   - no_mock

📂 Contextual:
   - region



# 5. Data Splitting 

In [19]:
#Define Features 
selected_features = [
    # Individual
    "gender", "life_sat",
    # Academic
    "math_hwork",
    # Family
    "SES", "num_sib", "food_sec", "family_stat", "has_computer", "has_books",
    # School
    "schl_belong", "teacher_help", "safe_student", "safe_class", "qual_math_instruct",
    # Peer/Social
    "make_friends", "feel_included", "no_mock",
    # Contextual
    "region"
]

In [20]:
feature_types = {
    # Individual
    "gender": "binary",
    "life_sat": "ordinal",

    # Academic
    "math_hwork": "ordinal",

    # Family
    "SES": "continuous",
    "num_sib": "discrete",
    "food_sec": "ordinal",
    "family_stat": "continuous",
    "has_computer": "binary",
    "has_books": "binary",

    # School
    "schl_belong": "ordinal",
    "teacher_help": "ordinal",
    "safe_student": "ordinal",
    "safe_class": "ordinal",
    "qual_math_instruct": "ordinal",

    # Peer/Social
    "make_friends": "ordinal",
    "feel_included": "ordinal",
    "no_mock": "ordinal",

    # Contextual
    "region": "categorical"
}

In [21]:
#Define Target 
target_col = "math_binary"

In [22]:
# Get weights
weights = df["W_FSTUWT"].copy()

In [23]:
# Create X, y, w
X = df[selected_features].copy()
y = df[target_col].copy()
w = weights.copy()

In [24]:
# Split into train / temp
X_temp, X_test, y_temp, y_test, w_temp, w_test = train_test_split(
    X, y, w, test_size=0.2, random_state=42
)

In [25]:
# Split train / val
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
    X_temp, y_temp, w_temp, test_size=0.25, random_state=42
)

In [26]:
print(f"Training:   {X_train.shape}")
print(f"Validation: {X_val.shape}")
print(f"Test:       {X_test.shape}")

Training:   (7782, 18)
Validation: (2595, 18)
Test:       (2595, 18)


# 6. Missing Data Analysis 

##### I am using simple imputation because it is fast, effective, and well-suited for decision trees. 
- Median imputation for all numeric features
- Mode imputation for the two binary categorical ones (has_computer, has_books)

In [27]:
def analyze_missing(df, name):
    print(f"\n📊 Missing value summary: {name}")
    missing = df.isna().sum()
    percent = df.isna().mean() * 100
    summary = pd.DataFrame({'Missing': missing, 'Percent': percent})
    summary = summary[summary['Missing'] > 0].sort_values('Percent', ascending=False)
    if summary.empty:
        print("✅ No missing values.")
    else:
        print(summary.round(2))
    return summary

In [28]:
train_missing = analyze_missing(X_train, "Train")
val_missing = analyze_missing(X_val, "Validation")
test_missing = analyze_missing(X_test, "Test")


📊 Missing value summary: Train
                    Missing  Percent
schl_belong            2234    28.71
make_friends           2185    28.08
math_hwork             1483    19.06
teacher_help           1443    18.54
qual_math_instruct     1290    16.58
family_stat            1165    14.97
SES                    1147    14.74
no_mock                1108    14.24
safe_student           1106    14.21
feel_included          1096    14.08
life_sat               1043    13.40
food_sec               1034    13.29
safe_class             1018    13.08
has_computer            976    12.54
has_books               933    11.99
num_sib                 927    11.91

📊 Missing value summary: Validation
                    Missing  Percent
make_friends            726    27.98
schl_belong             707    27.24
math_hwork              490    18.88
teacher_help            454    17.50
qual_math_instruct      421    16.22
family_stat             372    14.34
no_mock                 364    14.03
SES   

In [29]:
#define categorical and numeric cols
categorical_cols = ["has_computer", "has_books", "region"]

# Define numeric columns by checking actual dtypes in the dataframe
numeric_cols = [col for col in X_train.columns if col not in categorical_cols + ["target"]]

In [30]:
print("Missing values in categorical (binary) columns:")
print(X_train[categorical_cols].isna().sum())

print("\nMissing values in numeric columns:")
print(X_train[numeric_cols].isna().sum())

Missing values in categorical (binary) columns:
has_computer    976
has_books       933
region            0
dtype: int64

Missing values in numeric columns:
gender                   0
life_sat              1043
math_hwork            1483
SES                   1147
num_sib                927
food_sec              1034
family_stat           1165
schl_belong           2234
teacher_help          1443
safe_student          1106
safe_class            1018
qual_math_instruct    1290
make_friends          2185
feel_included         1096
no_mock               1108
dtype: int64


In [31]:
# Create clean working copies
X_train = X_train.copy()
X_val = X_val.copy()
X_test = X_test.copy()

In [32]:
# Median imputation for numeric
num_imputer = SimpleImputer(strategy="median")
X_train[numeric_cols] = num_imputer.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = num_imputer.transform(X_val[numeric_cols])
X_test[numeric_cols] = num_imputer.transform(X_test[numeric_cols])

# Mode imputation for binary categorical
cat_imputer = SimpleImputer(strategy="most_frequent")
X_train[categorical_cols] = cat_imputer.fit_transform(X_train[categorical_cols])
X_val[categorical_cols] = cat_imputer.transform(X_val[categorical_cols])
X_test[categorical_cols] = cat_imputer.transform(X_test[categorical_cols])

# 7. One-Hot Encode "region"

In [33]:
# One-hot encode region in train, val, test
X_train = pd.get_dummies(X_train, columns=["region"])
X_val = pd.get_dummies(X_val, columns=["region"])
X_test = pd.get_dummies(X_test, columns=["region"])

# Align columns across splits (to ensure same dummy variables exist)
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

categorical_cols = [col for col in X_train.columns if col.startswith("region_") or col in ["has_computer", "has_books"]]

# 8. Save to CSV with weights included 

In [34]:
# Add target and weights to each dataset
X_train["target"] = y_train
X_train["sample_weight"] = w_train

X_val["target"] = y_val
X_val["sample_weight"] = w_val

X_test["target"] = y_test
X_test["sample_weight"] = w_test

In [35]:
X_train.to_csv("/workspaces/mini_project_2/data/processed/train.csv", index=False)
X_val.to_csv("/workspaces/mini_project_2/data/processed/val.csv", index=False)
X_test.to_csv("/workspaces/mini_project_2/data/processed/test.csv", index=False)


In [36]:
X_train["target"] = y_train
X_val["target"] = y_val
X_test["target"] = y_test

X_train.to_csv("/workspaces/mini_project_2/data/processed/train.csv", index=False)
X_val.to_csv("/workspaces/mini_project_2/data/processed/val.csv", index=False)
X_test.to_csv("/workspaces/mini_project_2/data/processed/test.csv", index=False)

print("✅ Preprocessing complete. Data saved to /data/processed/")
print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

✅ Preprocessing complete. Data saved to /data/processed/
Train shape: (7782, 23)
Validation shape: (2595, 23)
Test shape: (2595, 23)


In [37]:
print(X_train.head())

       gender  life_sat  math_hwork     SES  num_sib  food_sec  family_stat  \
7371      0.0       4.0         1.0 -0.8355      4.0       3.0          4.0   
8058      0.0       7.0         1.0  0.1750      3.0       5.0          7.0   
12177     0.0       9.0         1.0 -0.5774      4.0       5.0          9.0   
4729      0.0       5.0         1.0 -0.3974      3.0       5.0          7.0   
1475      1.0       7.0         1.0  0.1750      3.0       5.0          7.0   

      has_computer has_books  schl_belong  ...  qual_math_instruct  \
7371           0.0       1.0          3.0  ...                 4.0   
8058           1.0       0.0          3.0  ...                 7.0   
12177          1.0       0.0          3.0  ...                 6.0   
4729           1.0       0.0          3.0  ...                 9.0   
1475           1.0       0.0          3.0  ...                 7.0   

       make_friends  feel_included  no_mock  region_england  region_n_ireland  \
7371            2.0    